In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bz2

In [2]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    labels = labels[:int(len(labels)*0.01)]
    texts = texts[:int(len(texts)*0.01)]
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('../input/amazonreviews/train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('../input/amazonreviews/test.ft.txt.bz2')

In [3]:
train_df=pd.DataFrame(zip(train_texts,train_labels),columns=['text','label'])
print(train_df.head())
test_df=pd.DataFrame(zip(test_texts,test_labels),columns=['text','label'])
print(test_df.head())

                                                text  label
0  Stuning even for the non-gamer: This sound tra...      1
1  The best soundtrack ever to anything.: I'm rea...      1
2  Amazing!: This soundtrack is my favorite music...      1
3  Excellent Soundtrack: I truly like this soundt...      1
4  Remember, Pull Your Jaw Off The Floor After He...      1
                                                text  label
0  Great CD: My lovely Pat has one of the GREAT v...      1
1  One of the best game music soundtracks - for a...      1
2  Batteries died within a year ...: I bought thi...      0
3  works fine, but Maha Energy is better: Check o...      1
4  Great for the non-audiophile: Reviewed quite a...      1


In [4]:
print(train_df.text[0])

Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^


In [5]:
import regex as re
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.regexp import RegexpStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [6]:
def remove_special_characters(text):
  text=text.str.lower()
  text=text.apply(lambda x: re.sub(r'[0-9]+','',x))
  text=text.apply(lambda x: re.sub(r'@mention',' ',x))
  text=text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ',x))
  text=text.apply(lambda x: re.sub(r"www.\[a-z]?\.?(com)+|[a-z]+\.(com)", ' ',x))
  text=text.apply(lambda x: re.sub(r"[_\,\>\(\-:\)\\\/\!\.\^\!\:\];='#]",'',x))
  return text

In [7]:
train_df['text']=remove_special_characters(train_df['text'])
test_df['text']=remove_special_characters(test_df['text'])

In [8]:
from keras.preprocessing import text,sequence

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 86207 unique tokens.


In [10]:
pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
from keras_preprocessing.sequence import pad_sequences

train_text = tokenizer.texts_to_sequences(train_df['text'].values)
train_text = pad_sequences(train_text, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', train_text.shape)

y = pd.get_dummies(train_df['label']).values
print('Shape of label tensor:', y.shape)

Shape of data tensor: (36000, 250)
Shape of label tensor: (36000, 2)


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(train_text,y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(32400, 250) (32400, 2)
(3600, 250) (3600, 2)


In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,SpatialDropout1D,GlobalMaxPooling1D, Dense
import tensorflow as tf

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=train_text.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(GlobalMaxPooling1D())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 250, 100)          1000000   
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 128)               12928     
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1,093,586
Trainable params: 1,093,586
Non-trainable params: 0
____________________________________________

In [18]:
from keras.callbacks import EarlyStopping
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
456/456 [==============================] - 538s 1s/step - loss: 0.4092 - accuracy: 0.8079 - val_loss: 0.2891 - val_accuracy: 0.8830
Epoch 2/5
456/456 [==============================] - 500s 1s/step - loss: 0.2374 - accuracy: 0.9089 - val_loss: 0.2867 - val_accuracy: 0.8815
Epoch 3/5
456/456 [==============================] - 489s 1s/step - loss: 0.1751 - accuracy: 0.9368 - val_loss: 0.3180 - val_accuracy: 0.8877
Epoch 4/5
456/456 [==============================] - 484s 1s/step - loss: 0.1339 - accuracy: 0.9528 - val_loss: 0.3971 - val_accuracy: 0.8895
Epoch 5/5
456/456 [==============================] - 484s 1s/step - loss: 0.1058 - accuracy: 0.9629 - val_loss: 0.4189 - val_accuracy: 0.8855


In [20]:
text = "I really enjoyed this movie. The acting was great and the plot was engaging. Highly recommend!"
# preprocess the text data
# text = preprocess_text(text)
text_sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(text_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(padded_sequence)
predicted_class = np.argmax(prediction)
sentiment = "positive" if predicted_class == 1 else "negative"
print("The sentiment of the text is:", sentiment)

1/1 [==============================] - 0s 338ms/step
The sentiment of the text is: positive


In [21]:
text = "The product was a complete waste of money. It broke within a week and the customer service was terrible."

text_sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(text_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(padded_sequence)
predicted_class = np.argmax(prediction)
sentiment = "positive" if predicted_class == 1 else "negative"
print("The sentiment of the text is:", sentiment)

1/1 [==============================] - 0s 82ms/step
The sentiment of the text is: negative


In [22]:
text = "I was really disappointed with this hotel. The room was dirty and the staff was unhelpful."

text_sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(text_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(padded_sequence)
predicted_class = np.argmax(prediction)
sentiment = "positive" if predicted_class == 1 else "negative"
print("The sentiment of the text is:", sentiment)

1/1 [==============================] - 0s 82ms/step
The sentiment of the text is: negative


In [26]:
model.save("/kaggle/working/model.h5")
model.save_weights("/kaggle/working/model_weights.h5")

In [27]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save the weights as an HDF5 file
model.save_weights("weights.h5")

# Save the weight manifest as JSON
import json
weight_manifest = {}
for layer in model.layers:
    if layer.weights:
        weight_manifest[layer.name] = [w.name for w in layer.weights]
with open("weight_manifest.json", "w") as json_file:
    json.dump(weight_manifest, json_file)